In [ ]:
# !pip install faiss-cpu
# !pip install gensim
# !pip install pybind11
# !pip install numpy<2


In [ ]:
import faiss # our agent to find connection between vectors
from gensim.models import Word2Vec # conversion words into vectors
import numpy as np # math

import nltk # to convert text into some more presentable way
nltk.download("punkt")
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize, word_tokenize

In [ ]:
def file_convert(file_name):
  with open(file_name, "r", encoding="utf-8") as file: ## reading a file and storing it into one variable
    f = file.read()

  sentenses = sent_tokenize(f) # our english teacher
  tokenized_sentenses = [word_tokenize(sent) for sent in sentenses] # structure a text

  model = Word2Vec(tokenized_sentenses, vector_size=100, window=5, min_count=2, sg=1) # convertation to vectors
  document_vectors = [] # TODO

  for sentence in tokenized_sentenses:
      sentence_vector = np.mean([model.wv[word] for word in sentence if word in model.wv], axis=0)
      if np.isnan(sentence_vector).any():
          sentence_vector = np.zeros(model.vector_size)
      document_vectors.append(sentence_vector)
  return document_vectors, model, tokenized_sentenses


In [ ]:
doc_vec, model, dataset = file_convert("church_text")
print(doc_vec[:3])
print(np.linalg.norm(doc_vec, axis=1))

# from sklearn.preprocessing import normalize -> 4 decimal difference (less than 1%)
# norm_vec = normalize(doc_vec, axis=1)
# print(norm_vec[:1])

In [ ]:
def faiss_search(document, model): # now we initialize a search on a given field
  document_vectors = np.array(document).astype("float32")

  index = faiss.IndexFlatIP(model.vector_size)
  index.add(document_vectors)  # add vectors to FAISS
  return index, model

In [ ]:
faiss_index, faiss_model = faiss_search(doc_vec ,model)

In [ ]:
### models part
import torch # pytorch
import pandas as pd # datasets
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [ ]:
phi = "microsoft/phi-2"
phi_model = AutoModelForCausalLM.from_pretrained(
    phi,
    torch_dtype=torch.float16, ## gpu support
    device_map="auto",
    trust_remote_code=True ## fix api issues
    )
phi_tokenizer = AutoTokenizer.from_pretrained(phi)

In [ ]:
question = "church robot"

sntc_question = sent_tokenize(question)
tokenized_question = [word_tokenize(sent) for sent in sntc_question]

# vectorization with numpy
words = [model.wv[word] for word in tokenized_question[0] if word in model.wv]

if not words:
    question_vector = np.zeros(model.vector_size)
else:
    question_vector = np.mean(words, axis=0)

query = question_vector.reshape(1, -1).astype("float32")
query[:1]

In [ ]:
# faiss search
distances, indices = faiss_index.search(query, k=3)

threshold = 0.2 # ограничение по "релевантности" запроса, насколько соотвествует ожиданиям, сортировка мусора

if distances[0][0] < threshold:
  print("trash values")
else:
  for idx in indices[0]:
    print(dataset[idx])

